In [1]:
import pandas as pd
from myUtils import pickle_load, symb_perf_stats_vectorized

file_close = 'df_close_clean'
path_dir = "C:/Users/ping/MyDrive/stocks/yfinance"
path_data_dump = path_dir + "/VSCode_dump/"

df_c = pickle_load(path_data_dump, file_close)
df_c = df_c[-252::]
format1 = 'df_c({}):\n{}\n{}'.format(len(df_c), df_c.head(3), df_c.tail(3))
# print(format1)

In [2]:
def perf_eval(df_close):
  '''
  df_close is a dataframe with date index, columns of symbols' closing price, and symbol as column name 
  '''
  # %%timeit
  symbols, period_yr, drawdown, UI, max_drawdown, returns_std, Std_UI, CAGR, CAGR_Std, CAGR_UI = \
      symb_perf_stats_vectorized(df_close)
  caches_perf_stats_vect = []
  for symbol in symbols:
      date_first = drawdown.index[0].strftime('%Y-%m-%d')
      date_last = drawdown.index[-1].strftime('%Y-%m-%d')
      cache = (symbol, date_first, date_last, period_yr, CAGR[symbol],
              UI[symbol], Std_UI[symbol], CAGR_Std[symbol], CAGR_UI[symbol])
      # append performance data (tuple) to caches_perf_stats (list)
      caches_perf_stats_vect.append(cache)
  column_names = ['symbol', 'first date', 'last date', 'Year', 'CAGR',
                  'UI', 'Std/UI', 'CAGR/Std', 'CAGR/UI']
  # write symbols' performance stats to dataframe
  df_perf = pd.DataFrame(caches_perf_stats_vect, columns=column_names)

  _cols = ['CAGR', 'UI', 'Std/UI', 'CAGR/Std', 'CAGR/UI']
  # print(f'df_perf mean:\n{df_perf[_cols].mean()}\n')
  # print(f'df_perf std:\n{df_perf[_cols].std()}')
  grp_CAGRUI_mean = df_perf['CAGR/UI'].mean()
  grp_CAGRUI_std = df_perf['CAGR/UI'].std()
  grp_CAGRUI_mean_std = grp_CAGRUI_mean / grp_CAGRUI_std
  grp_CAGRUI_mean_std

  grp_CAGRStd_mean = df_perf['CAGR/Std'].mean()
  grp_CAGRStd_std = df_perf['CAGR/Std'].std()
  grp_CAGRStd_mean_std = grp_CAGRStd_mean / grp_CAGRStd_std
  grp_CAGRStd_mean_std

  grp_StdUI_mean = df_perf['Std/UI'].mean()
  grp_StdUI_std = df_perf['Std/UI'].std()
  grp_StdUI_mean_std = grp_StdUI_mean / grp_StdUI_std
  grp_StdUI_mean_std

  return df_perf, grp_StdUI_mean_std, grp_CAGRStd_mean_std, grp_CAGRUI_mean_std

In [3]:
df_perf, grp_StdUI_mean_std, grp_CAGRStd_mean_std, grp_CAGRUI_mean_std = perf_eval(df_c)
print(f'grp_StdUI_mean_std :{grp_StdUI_mean_std:.6f}')
print(f'grp_CAGRStd_mean_std :{grp_CAGRStd_mean_std:.6f}')
print(f'grp_CAGRUI_mean_std :{grp_CAGRUI_mean_std:.6f}')
df_perf.head()

grp_StdUI_mean_std :2.074698
grp_CAGRStd_mean_std :-0.126652
grp_CAGRUI_mean_std :0.165209


,symbol,first date,last date,Year,CAGR,UI,Std/UI,CAGR/Std,CAGR/UI
0,A,2021-11-26,2022-11-25,1.0,0.038767,0.182132,0.121777,1.747853,0.212848
1,AA,2021-11-26,2022-11-25,1.0,0.006143,0.380503,0.109612,0.147291,0.016145
2,AAL,2021-11-26,2022-11-25,1.0,-0.183099,0.238084,0.151246,-5.084774,-0.769049
3,AAON,2021-11-26,2022-11-25,1.0,0.012637,0.292647,0.083641,0.516295,0.043183
4,AAP,2021-11-26,2022-11-25,1.0,-0.318919,0.193532,0.122843,-13.414544,-1.647886


In [4]:
# _cols2 = ['TSBK', 'ELF', 'AVEO', 'ASC', 'TPL', 'AXON', 'NRIM', 'AAON', 'UFPT', 'GILD', 'MOD', 'SANM', 'INSW', 'LOPE', 'AE', 'ABMD', 'RMBS', 'WNC']
_cols2 = ['AE', 'ABMD', 'RMBS', 'WNC']
df_temp = df_c[_cols2].copy()
# df_perf, grp_CAGRUI_mean_std = perf_eval(df_temp)
df_perf, grp_StdUI_mean_std, grp_CAGRStd_mean_std, grp_CAGRUI_mean_std = perf_eval(df_temp)
# print(f'grp_CAGRUI_mean_std :{grp_CAGRUI_mean_std:.6f}')
print(f'grp_StdUI_mean_std: {grp_StdUI_mean_std:.6f}')
print(f'grp_CAGRStd_mean_std: {grp_CAGRStd_mean_std:.6f}')
print(f'grp_CAGRUI_mean_std: {grp_CAGRUI_mean_std:.6f}')
df_perf.head()

grp_StdUI_mean_std: 4.485206
grp_CAGRStd_mean_std: 2.066832
grp_CAGRUI_mean_std: 1.924932


,symbol,first date,last date,Year,CAGR,UI,Std/UI,CAGR/Std,CAGR/UI
0,AE,2021-11-26,2022-11-25,1.0,0.436282,0.135952,0.181407,17.690065,3.209093
1,ABMD,2021-11-26,2022-11-25,1.0,0.159826,0.226057,0.189059,3.739657,0.707015
2,RMBS,2021-11-26,2022-11-25,1.0,0.401202,0.202183,0.129537,15.318777,1.984351
3,WNC,2021-11-26,2022-11-25,1.0,0.476516,0.242482,0.121844,16.128516,1.965157


In [5]:
df_SPY = df_c[['SPY']].copy()
df_SPY
df_perf, grp_StdUI_mean_std, grp_CAGRStd_mean_std, grp_CAGRUI_mean_std = perf_eval(df_SPY)
print(f'grp_StdUI_mean_std: {grp_StdUI_mean_std:.6f}')
print(f'grp_CAGRStd_mean_std: {grp_CAGRStd_mean_std:.6f}')
print(f'grp_CAGRUI_mean_std: {grp_CAGRUI_mean_std:.6f}')
df_perf.head()

grp_StdUI_mean_std: nan
grp_CAGRStd_mean_std: nan
grp_CAGRUI_mean_std: nan


,symbol,first date,last date,Year,CAGR,UI,Std/UI,CAGR/Std,CAGR/UI
0,SPY,2021-11-26,2022-11-25,1.0,-0.110142,0.14084,0.107612,-7.267189,-0.782035
